In [2]:
pip install simpy

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\oihater\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [13]:
import simpy
import random

# 시간 단위 : μs
# DataFrame = 1000byte, 각 station 전송속도는 항상 80Mbps 가정 -> 한개의 데이터프레임 당 100μs 소모(Transmission Delay)

cnt_load = []  # count load
cnt_succeed = []  # count transmission success

# datalength = random.randint(10, 200) -> 데이터크기 100byte~2000byte랜덤으로 생성, 전송시간 10~200μs

# 캐리어 센싱
def CarrierSensing(que):
    # idle
    if not que: #채널 큐가 비어있는지 비어있으면 idle 아니면 busy를 반환
        return 'idle'

    # busy
    else:
        return 'busy'

    
class Station:
    
    staID = 1  
    CS = ''

    def __init__(self, env):

        self.staID = Station.staID
        self.CS = Station.CS
        Station.staID += 1
        self.env = env
        env.process(self.run())
        
        
    def run(self):
        arrivalRate=0.01  # 평균 100μs 당 1개의 Data
        DataSend= random.expovariate(arrivalRate)  # 평균 100을 갖는 랜덤 포아송, 첫 데이터 전송시점
        trans_time=0.0
        
        # station begins
        while True:
            datalength = random.randint(100, 1000) # 데이터 길이 -> 100~1000byte
            trans_time = datalength*8 / 80  # 전송시간, 전송속도 80Mbps로 고정
            yield env.timeout(DataSend)  # 데이터 전송시 채널 시점
            
            #캐리어 센싱
            self.CS = CarrierSensing(Channel.que) 
            print('Station %d : ' %self.staID, 'Carrier Sensing at : %f' %self.env.now, "Result : ", self.CS)
            
            cnt_load.append(self.staID)  # 데이터 도착 count
            
            if self.CS == 'busy':
                while True:
                    backoff = random.expovariate(1/trans_time)  # 랜덤시간 대기
                    yield env.timeout(backoff)
                    print('Station %d : ' %self.staID, 'back off to : %f' %self.env.now)

                    self.CS = CarrierSensing(Channel.que)  # 캐리어 센싱
                    print('Station %d : ' %self.staID, 'Carrier Sensing again at : %f' %self.env.now, "Result : ", self.CS)

                    if self.CS == 'busy':
                        continue

                    elif self.CS == 'idle':
                        while True:
                            backoff = random.uniform(0, 1)  # IFS
                            yield env.timeout(backoff)
                            print('Station %d : ' %self.staID, 'wait for IFS to : %f' %self.env.now)
                            
                            self.CS = CarrierSensing(Channel.que)
                            if self.CS == 'idle':
                                Channel.que.append(self.staID)
                                print('Station %d : ' %self.staID, 'Transmission start at : %f' %self.env.now)
                                yield env.timeout(trans_time)

                                print('Station %d : ' %self.staID, 'Transmission complete(ACK) at : %f' %self.env.now)
                                cnt_succeed.append(self.staID)
                                Channel.que.clear()
                                break
                        break    
                            

            
            elif self.CS == 'idle':
                while True:
                    backoff = random.uniform(0, 1)  # IFS
                    yield env.timeout(backoff)
                    print('Station %d : ' %self.staID, 'wait for IFS to : %f' %self.env.now)

                    self.CS = CarrierSensing(Channel.que)
                    print('Station %d : ' %self.staID, 'Carrier Sensing again at : %f' %self.env.now, "Result : ", self.CS)
                    if self.CS == 'idle':
                        Channel.que.append(self.staID)
                        print('Station %d : ' %self.staID, 'Transmission start at : %f' %self.env.now)
                        yield env.timeout(trans_time)

                        print('Station %d : ' %self.staID, 'Transmission complete(ACK) at : %f' %self.env.now)
                        cnt_succeed.append(self.staID)
                        Channel.que.clear()
                        break

                    elif self.CS == 'busy':
                        
                        backoff = random.expovariate(1/trans_time)  # 랜덤시간 대기
                        yield env.timeout(backoff)
                        print('Station %d : ' %self.staID, 'back off to : %f' %self.env.now)

                        self.CS = CarrierSensing(Channel.que)  # 캐리어 센싱
                        print('Station %d : ' %self.staID, 'Carrier Sensing at : %f' %self.env.now, "Result : ", self.CS)
            
            DataSend = random.expovariate(arrivalRate)  # 다음 데이터 전송시점   
                      
# channel
class Channel:
    que = []

    def __init__(self, env):
        Channel.succeedMsgEvt = env.event()
        Channel.failMsgEvt = env.event()
        Channel.idleMsgEvt = env.event()
        Channel.colCount = 0
        self.env = env
        env.process(self.run())

    def run(self):
        while True:
            Channel.nowtime += SIFsTime
            yield self.env.timeout(SIFsTime)   


env = simpy.Environment()
numofStation = 5 #스테이션 수
staSet = [Station(env) for i in range(numofStation)]
env.run(until=1000)

load = 0.0
for i in range(len(cnt_load)):
    load += 1
                                           
throughput = 0.0
for i in range(len(cnt_succeed)):
    throughput += 1                                           
                                          
print('throughput : ', throughput/load)

Station 1 :  Carrier Sensing at : 7.919139 Result :  idle
Station 1 :  wait for IFS to : 8.408760
Station 1 :  Carrier Sensing again at : 8.408760 Result :  idle
Station 1 :  Transmission start at : 8.408760
Station 5 :  Carrier Sensing at : 27.919138 Result :  busy
Station 1 :  Transmission complete(ACK) at : 55.808760
Station 3 :  Carrier Sensing at : 100.566648 Result :  idle
Station 3 :  wait for IFS to : 100.605951
Station 3 :  Carrier Sensing again at : 100.605951 Result :  idle
Station 3 :  Transmission start at : 100.605951
Station 1 :  Carrier Sensing at : 127.368204 Result :  busy
Station 5 :  back off to : 177.940759
Station 5 :  Carrier Sensing again at : 177.940759 Result :  busy
Station 2 :  Carrier Sensing at : 180.085052 Result :  busy
Station 3 :  Transmission complete(ACK) at : 193.905951
Station 3 :  Carrier Sensing at : 196.555263 Result :  idle
Station 3 :  wait for IFS to : 197.505849
Station 3 :  Carrier Sensing again at : 197.505849 Result :  idle
Station 3 :  T

In [1]:
0.019/0.022

0.8636363636363636